# Station/DUID/Genset check

I want to check the heirachy

In [8]:
from nemosis import defaults, dynamic_data_compiler

In [14]:
start_time = '2025/10/01 00:00:00'
end_time = '2025/10/02 00:05:00'
raw_data_cache = '/home/matthew/Data/nemosis'

In [16]:
tables = ["DUDETAILS", "DUDETAIL", "DUALLOC", "STATION", "STATIONOWNER"]
data = {}
for table in tables:
    print(f"Getting table {table}")
    data[table] = dynamic_data_compiler(start_time, end_time, table, raw_data_cache)

Getting table DUDETAILS


UserInputError: Table name provided is not a dynamic table.

In [17]:
import polars as pl

In [18]:
df = pl.read_csv("~/Downloads/stations.csv")
df

STATIONNAME,DUID,GENSETID
str,str,str
"""Appin Power Plant""","""APPIN""","""APPIN"""
"""Avonlie Solar Farm""","""AVLSF1""","""AVLSF1"""
"""Bango 973 Wind Farm""","""BANGOWF1""","""BANGOWF1"""
"""Bango 999 Wind Farm""","""BANGOWF2""","""BANGOWF2"""
"""Burrendong Hydro Power Station""","""BDONGHYD""","""BDONGHYD"""
…,…,…
"""YALLOURN 'W' POWER STATION""","""YWNL1""","""YWNL1"""
"""YALLOURN 'W' POWER STATION""","""YWPS1""","""YWPS1"""
"""YALLOURN 'W' POWER STATION""","""YWPS2""","""YWPS2"""


In [20]:
# STATIONNAME -> DUID relationships
station_to_duid = df.group_by('STATIONNAME').agg(
    pl.col('DUID').unique().alias('DUIDs'),
    pl.col('DUID').n_unique().alias('count')
)
station_multi_duid = station_to_duid.filter(pl.col('count') > 1)

# DUID -> STATIONNAME relationships
duid_to_station = df.group_by('DUID').agg(
    pl.col('STATIONNAME').unique().alias('STATIONNAMEs'),
    pl.col('STATIONNAME').n_unique().alias('count')
)
duid_multi_station = duid_to_station.filter(pl.col('count') > 1)

# DUID -> GENSETID relationships
duid_to_genset = df.group_by('DUID').agg(
    pl.col('GENSETID').unique().alias('GENSETIDs'),
    pl.col('GENSETID').n_unique().alias('count')
)
duid_multi_genset = duid_to_genset.filter(pl.col('count') > 1)

# GENSETID -> DUID relationships
genset_to_duid = df.group_by('GENSETID').agg(
    pl.col('DUID').unique().alias('DUIDs'),
    pl.col('DUID').n_unique().alias('count')
)
genset_multi_duid = genset_to_duid.filter(pl.col('count') > 1)

print("=" * 80)
print("HIERARCHY ANALYSIS")
print("=" * 80)
print(f"\nTotal records: {len(df)}")
print(f"Unique STATIONNAMEs: {df['STATIONNAME'].n_unique()}")
print(f"Unique DUIDs: {df['DUID'].n_unique()}")
print(f"Unique GENSETIDs: {df['GENSETID'].n_unique()}")

print("\n" + "=" * 80)
print("DUID -> STATIONNAME")
print("=" * 80)
if len(duid_multi_station) == 0:
    print("✓ One-to-one: Each DUID maps to exactly one STATIONNAME")
else:
    print("✗ Multiple STATIONNAMEs per DUID:")
    print(duid_multi_station)

print("\n" + "=" * 80)
print("STATIONNAME -> DUID")
print("=" * 80)
if len(station_multi_duid) == 0:
    print("✓ One-to-one: Each STATIONNAME maps to exactly one DUID")
else:
    print("✗ Multiple DUIDs per STATIONNAME:")
    print(station_multi_duid)

print("\n" + "=" * 80)
print("DUID -> GENSETID")
print("=" * 80)
if len(duid_multi_genset) == 0:
    print("✓ One-to-one: Each DUID maps to exactly one GENSETID")
else:
    print(f"⚠ Multiple GENSETIDs per DUID ({len(duid_multi_genset)} DUIDs):")
    print(duid_multi_genset.head(10))
    if len(duid_multi_genset) > 10:
        print(f"... and {len(duid_multi_genset) - 10} more")

print("\n" + "=" * 80)
print("GENSETID -> DUID")
print("=" * 80)
if len(genset_multi_duid) == 0:
    print("✓ One-to-one: Each GENSETID maps to exactly one DUID")
else:
    print("✗ Multiple DUIDs per GENSETID:")
    print(genset_multi_duid)

print("\n" + "=" * 80)
print("CONCLUSION")
print("=" * 80)
print("Hierarchy: STATIONNAME ↔ DUID ← GENSETID")
print("           (1:1)          (1:many)")

HIERARCHY ANALYSIS

Total records: 605
Unique STATIONNAMEs: 420
Unique DUIDs: 549
Unique GENSETIDs: 605

DUID -> STATIONNAME
✓ One-to-one: Each DUID maps to exactly one STATIONNAME

STATIONNAME -> DUID
✗ Multiple DUIDs per STATIONNAME:
shape: (66, 3)
┌─────────────────────────────────┬─────────────────────────────────┬───────┐
│ STATIONNAME                     ┆ DUIDs                           ┆ count │
│ ---                             ┆ ---                             ┆ ---   │
│ str                             ┆ list[str]                       ┆ u32   │
╞═════════════════════════════════╪═════════════════════════════════╪═══════╡
│ Lucas Heights 2 Power Station   ┆ ["LUCAS2S2", "LUCASHGT"]        ┆ 2     │
│ Laverton North Power Station    ┆ ["LNGS2", "LNGS1"]              ┆ 2     │
│ Uranquinty Power Station        ┆ ["URANQ11", "URANQ13", … "URAN… ┆ 4     │
│ LOY YANG B POWER STATION        ┆ ["LOYYB2", "LOYYB1"]            ┆ 2     │
│ Vales Point "B" Power Station   ┆ ["VP6", "VP

In [26]:
(
    df
    .filter(pl.col("DUID") == "COOPGWF1")
)

STATIONNAME,DUID,GENSETID
str,str,str
"""Coopers Gap Wind Farm""","""COOPGWF1""","""COOPGWF1"""
"""Coopers Gap Wind Farm""","""COOPGWF1""","""COOPGWF2"""
